In [21]:
import pyproj
import math
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import ORBSLAMResults, umeyama_alignment, fit_trajectory_nn, predict_trajectory_nn

results = ORBSLAMResults("~/msc/shared_data/orbslam-out-utm-nn-vabadusepst")

gps_trajectory_wgs = pd.DataFrame([(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                              for kf in results.keyframes[1:]], columns=['lat', 'lon', 'alt'])
slam_trajectory = np.array([(kf.x, kf.y, kf.z) for kf in results.keyframes[1:]])

# Create transformers for WGS84 <-> UTM35N
wgs2utm = pyproj.Transformer.from_crs(4326, 32635)
utm2wgs = pyproj.Transformer.from_crs(32635, 4326)

# Convert GPS trajectory (WGS84) to UTM35N
gps_trajectory_utm = np.array([wgs2utm.transform(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                                   for kf in results.keyframes[1:]])

# Align SLAM trajectory to GPS trajectory
R, t, c  = umeyama_alignment(slam_trajectory.T, gps_trajectory_utm.T, True)
aligned_slam_trajectory_utm = np.array([t + c * R @ p for p in slam_trajectory])

# Convert SLAM trajectory (UTM35N) to WGS84
aligned_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2]) for p in aligned_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

model = fit_trajectory_nn(aligned_slam_trajectory_utm, gps_trajectory_utm, epochs=400)

fitted_slam_trajectory_utm = predict_trajectory_nn(model, aligned_slam_trajectory_utm)
fitted_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2]) for p in fitted_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

slam_estimates = np.array([(e.lat, e.lon, e.alt) for e in results.slam_estimates])
slam_estimates_samples = slam_estimates[sorted(np.random.default_rng().choice(range(len(slam_estimates)), size=len(slam_trajectory), replace=False))]
aligned_slam_estimate_utm = np.array([t + c * R @ p for p in slam_estimates_samples])
fitted_slam_estimate_utm = predict_trajectory_nn(model, aligned_slam_estimate_utm)
fitted_slam_estimate_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2]) for p in fitted_slam_estimate_utm], columns=['lat', 'lon', 'alt'])

fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(lat=gps_trajectory_wgs['lat'], 
                     lon=gps_trajectory_wgs['lon'], 
                     mode='markers+lines',
                     marker=dict(color='blue'),
                     name='GPS'))
fig.add_trace(
    go.Scattermapbox(lat=aligned_slam_trajectory_wgs['lat'], 
                     lon=aligned_slam_trajectory_wgs['lon'], 
                     mode='markers+lines', 
                     marker=dict(color='red'),
                     name='SLAM'))
fig.add_trace(
    go.Scattermapbox(lat=fitted_slam_trajectory_wgs['lat'], 
                     lon=fitted_slam_trajectory_wgs['lon'], 
                     mode='markers+lines', 
                     marker=dict(color='forestgreen'),
                     name='fitted SLAM'))
fig.add_trace(
    go.Scattermapbox(lat=fitted_slam_estimate_wgs['lat'], 
                     lon=fitted_slam_estimate_wgs['lon'], 
                     mode='markers+lines', 
                     name='fitted SLAM estimate'))
fig.update_geos(projection_type="transverse mercator")
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=np.mean(gps_trajectory_wgs['lat']), lon=np.mean(gps_trajectory_wgs['lon'])), zoom=15),
                  margin={"t": 0, "b": 0, "l": 0, "r": 0}, 
                  height=800)
fig.show()

Epoch 1/400
1/1 [==============================] - 0s 271ms/step - loss: 15832877891584.0000
Epoch 2/400
1/1 [==============================] - 0s 11ms/step - loss: 13280222380032.0000
Epoch 3/400
1/1 [==============================] - 0s 5ms/step - loss: 11979282448384.0000
Epoch 4/400
1/1 [==============================] - 0s 12ms/step - loss: 10252512657408.0000
Epoch 5/400
1/1 [==============================] - 0s 12ms/step - loss: 8089179258880.0000
Epoch 6/400
1/1 [==============================] - 0s 6ms/step - loss: 5869547814912.0000
Epoch 7/400
1/1 [==============================] - 0s 5ms/step - loss: 4172292030464.0000
Epoch 8/400
1/1 [==============================] - 0s 4ms/step - loss: 3270804242432.0000
Epoch 9/400
1/1 [==============================] - 0s 4ms/step - loss: 2551936450560.0000
Epoch 10/400
1/1 [==============================] - 0s 4ms/step - loss: 1707937759232.0000
Epoch 11/400
1/1 [==============================] - 0s 5ms/step - loss: 1029379391488.0000